In [ ]:
!pip install -U --quiet \
    "pinecone" \
    "langchain-pinecone" \
    "langchain-openai" \
    "langchain-ollama" \
    "langchain-community" \
    "langchain-core" \
    "langchain" \
    "langchain-text-splitters" \
    "dotenv" \
    "pypdf"

In [1]:

import os
from dotenv import load_dotenv

# pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone.vectorstores import PineconeVectorStore 

# chunking functions
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# llm and embeddings
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import ChatOllama  

# langchain rag
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


/Users/abhishekkakolla/code/curve/rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/abhishekkakolla/code/curve/rag/.venv/lib/python3.11/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVector

In [2]:
# api keys
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not PINECONE_API_KEY or not OPENAI_API_KEY:
    print("Error: API keys not found. Please create a .env file.")
else:
    print("API keys loaded successfully.")


API keys loaded successfully.


In [3]:
# connect to pinecone
INDEX_NAME = "vanilla-rag-ollama"
EMBEDDING_DIMENSION = 1536 # Dimension for OpenAI's text-embedding-3-small

pc = Pinecone(api_key=PINECONE_API_KEY)

# Create the index if it doesn't exist
if INDEX_NAME not in pc.list_indexes().names():
    print(f"Creating index '{INDEX_NAME}'...")
    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBEDDING_DIMENSION,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print("Index created.")
else:
    print(f"Index '{INDEX_NAME}' already exists.")


Index 'vanilla-rag-ollama' already exists.


### Ingestion

In [ ]:
# Ingestion
print("Loading documents from '12_papers'...")
loader = DirectoryLoader(
    '12_papers',                  # The folder to load from
    glob="**/*.pdf",              # Pattern to match only PDF files
    loader_cls=PyPDFLoader,       # Use PyPDFLoader to read the files
    show_progress=True
)
raw_docs = loader.load()

if not raw_docs:
    print("Error: No PDF files found in '12_papers'. Make sure they are in the correct directory.")
else:
    print(f"Loaded {len(raw_docs)} documents.")

# split into chunk
print("Splitting documents into chunks...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  
    chunk_overlap=200
)
all_splits = text_splitter.split_documents(raw_docs)
print(f"Split into {len(all_splits)} chunks.")

# embed models
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
)

print("Embedding and ingesting chunks..")
docsearch = PineconeVectorStore.from_documents(  
    documents=all_splits,
    embedding=embeddings,
    index_name=INDEX_NAME
)
print("Ingestion complete")



### Query

In [6]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
)

docsearch = PineconeVectorStore(
    index_name=INDEX_NAME,
    embedding=embeddings
)

In [7]:
# Initialize the LLM
llm = ChatOllama(
    model=os.getenv("FLASH_LLM"), # Use env var, but default to gemma2:9b
    base_url=os.getenv("OLLAMA_BASE_URL", "http://localhost:11434"),
    temperature=0.3,
)

# Pull the standard RAG prompti
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

# create chain
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

# retriever
retriever = docsearch.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)


### Question

In [8]:
import textwrap
question = "What do astrocytes contribute to Alzheimer's?" 
print(f"\nQuery: {question}")

# Invoke the chain
response = retrieval_chain.invoke({"input": question})
print("\nAnswer:")
ans = response["answer"]
wrapped_text = textwrap.fill(ans, width=70)
print(wrapped_text)


Query: What do astrocytes contribute to Alzheimer's?

Answer:
Astrocytes contribute to Alzheimer's disease by displaying fragmented
morphology, which can lead to neurotoxicity. Additionally, they are
involved in the clearance of plaque buildup and other cellular debris.
Reactive A1 astrocytes induced by microglial cytokines become toxic
and directly contribute to neuronal death in neurodegenerative
diseases, highlighting astrocyte regulation as a therapeutic target in
Alzheimer's disease.
